In [1]:
import numpy as np

In [72]:
#hyperparameters for ADR

## Number of rows
N=5
## Number of input neurons
K=128
## Nb of outputs, i.e. hidden nodes
M=128
##length of a sub-vector
neuronPerRow=32
## amount of hashing functions
H=3

In [73]:
L=K//neuronPerRow

In [74]:
# Create some data

X=np.random.normal(size=(N,K))
W=np.random.normal(size=(K,M))

In [75]:
#Create hashing vectors
hashingVectors=np.random.normal(size=(L,H))
hashingVectors/=np.sum(hashingVectors,axis=0,keepdims=True)

In [76]:
#Build XInput, the real ADR input
neuronXInput=[]
clusterMapping=np.zeros((N,K//L),dtype=np.uint8)

for l in range(K//L):
    localInputMatrix=[]
    clusterIds=dict()
    counts=[]
    for n in range(N):
        neuron=X[n,l*L:(l+1)*L]
        clusterBitId=np.matmul(neuron,hashingVectors)>0
        cNum=np.packbits(clusterBitId.astype(np.uint8),bitorder="little")[0]
        if cNum not in clusterIds.keys():
            clusterIds[cNum]=len(localInputMatrix)
            localInputMatrix.append(np.zeros((1,L)))
            counts.append(0)
        localInputMatrix[clusterIds[cNum]]+=neuron
        counts[clusterIds[cNum]]+=1
        clusterMapping[n,l]=clusterIds[cNum]
    print(counts)
    for i,count in enumerate(counts):
        localInputMatrix[i]/=count
    neuronXInput.append(np.concatenate(localInputMatrix,axis=0))

[1, 1, 2, 1]
[1, 1, 1, 1, 1]
[1, 2, 1, 1]
[1, 1, 1, 1, 1]
[1, 2, 1, 1]
[1, 1, 1, 1, 1]
[1, 3, 1]
[2, 1, 1, 1]
[1, 1, 3]
[1, 2, 1, 1]
[2, 1, 1, 1]
[2, 1, 1, 1]
[1, 2, 2]
[1, 3, 1]
[1, 1, 1, 2]
[2, 1, 1, 1]
[3, 1, 1]
[2, 1, 1, 1]
[1, 3, 1]
[2, 1, 2]
[1, 1, 2, 1]
[2, 2, 1]
[3, 1, 1]
[2, 2, 1]
[1, 1, 1, 1, 1]
[1, 2, 1, 1]
[1, 2, 2]
[3, 1, 1]
[1, 2, 1, 1]
[2, 2, 1]
[2, 1, 1, 1]
[3, 1, 1]


In [77]:
neuronYOutput=[]
for i, XInput in enumerate(neuronXInput):
    neuronYOutput.append(XInput@W[L*i:L*(i+1),:])

In [78]:
for i in range(len(neuronXInput)):
    print(i, " : ",neuronXInput[i].shape[0], "clusters")

0  :  4 clusters
1  :  5 clusters
2  :  4 clusters
3  :  5 clusters
4  :  4 clusters
5  :  5 clusters
6  :  3 clusters
7  :  4 clusters
8  :  3 clusters
9  :  4 clusters
10  :  4 clusters
11  :  4 clusters
12  :  3 clusters
13  :  3 clusters
14  :  4 clusters
15  :  4 clusters
16  :  3 clusters
17  :  4 clusters
18  :  3 clusters
19  :  3 clusters
20  :  4 clusters
21  :  3 clusters
22  :  3 clusters
23  :  3 clusters
24  :  5 clusters
25  :  4 clusters
26  :  3 clusters
27  :  3 clusters
28  :  4 clusters
29  :  3 clusters
30  :  4 clusters
31  :  3 clusters


In [79]:
neuronYOutput[0].shape

(4, 128)

In [80]:
Y=sum([np.concatenate([neuronYOutput[i][clusterMapping[n,i]].reshape((1,-1))
            for n in range(N)
        ],axis=0)
    for i in range(len(neuronYOutput))])

In [81]:
realY=X@W

In [82]:
relativeError=np.abs(Y-realY)/np.maximum(1e-6,np.abs(Y)+np.abs(realY))

In [83]:
print("Max rel error : %.2f, min rel error : %.2f, average rel error : %.2f" % (relativeError.max(),relativeError.min(),relativeError.mean()))

Max rel error : 1.00, min rel error : 0.00, average rel error : 0.35


In [84]:
relativeError

array([[1.19715487e-01, 7.04496393e-01, 2.67333540e-01, 1.00000000e+00,
        6.37943773e-01, 4.03438329e-01, 1.71836059e-01, 1.44955450e-02,
        5.98530831e-03, 1.89639404e-01, 1.68972491e-01, 1.50363829e-01,
        2.53141669e-01, 7.84700958e-01, 2.35427288e-01, 3.21789634e-01,
        1.75092482e-01, 4.18833240e-02, 7.39473778e-02, 9.65032282e-02,
        1.00610226e-01, 6.89174547e-03, 4.24850672e-01, 5.73395146e-01,
        2.51213458e-01, 8.41347048e-03, 8.67730480e-01, 2.42657644e-02,
        3.85013883e-01, 1.32424045e-01, 1.01441036e-01, 5.83528234e-02,
        2.55406476e-01, 1.00000000e+00, 7.60506676e-02, 7.43702301e-02,
        1.44683868e-02, 5.51028353e-01, 1.87516778e-02, 1.44038770e-01,
        1.75236098e-02, 1.00000000e+00, 2.73793334e-02, 1.00000000e+00,
        2.65297370e-01, 4.07284547e-02, 3.70926821e-02, 1.43071615e-01,
        5.81705976e-01, 2.31364586e-01, 4.26887490e-02, 1.28587313e-01,
        5.22537399e-01, 1.00000000e+00, 2.25916542e-01, 1.942419

In [85]:
np.sum(np.abs(realY-Y))/realY.size

3.492393663471425

Take two vectors as example and verify they have the same hashing